In [8]:
import random
import pygame
from os.path import join, expanduser
from datetime import datetime
import sys
sys.path.append("..")
from utility import DataLogger

In [9]:

dlog = DataLogger()

# unpickle a few files from Downloads
path_to_files = join(expanduser("~"), "Downloads", "inverted_pendulum")
files = os.listdir(path_to_files)
print(files)


['01-02-22_16-50', '01-02-22_16-54', '01-02-22_16-55', '01-02-22_16-58', '01-02-22_16-57', '01-02-22_16-53', '01-02-22_16-52']


In [12]:
help(pygame.image.save_extended)

Help on built-in function save_extended in module pygame.image:

save_extended(...)
    save_extended(Surface, filename) -> None
    save_extended(Surface, fileobj, namehint=) -> None
    save a png/jpg image to file (or file-like object)



In [5]:
a = 'condition_{:0>2}.jpg'.format(1)
a

'condition_01.jpg'

In [8]:
a = f'condition_{1:0>2}.jpg'
a

'condition_01.jpg'

In [14]:
import h5py
import numpy as np

h5File = h5py.File("test.hdf5", 'a')
grp = h5File.create_group("itr_0")
cond = grp.create_dataset("condition_0/image_1", data=np.arange(12).reshape(3, 4))

In [46]:
path_to_file = "experiments/inverted_pendulum/data_files/iter_0.hdf5"
# key = unicode('Annotation', "utf-8")
cond_0
def see(n, obj):
    print(n)
    for k, v in obj.attrs.items():
        print(k, v)

with h5py.File(path_to_file, 'r')  as df:

    cond_0 = df["condition_00"]
    cond_0.visititems(see)
    # print(cond_0)

RuntimeError: Object visitation failed (addr overflow, addr = 2536, size = 328, eoa = 2048)